# Add New Flowcells to Sample Table
## Author: Jonn Smith
## Date: 2023/03/09

Updates the sample table with any new flowcells from the flowcell table.

This script only adds new flowcells to the sample table.  It does not modify the sample table in any other way.

***

In [2]:
import os
import json
import math
import time

import firecloud.api as fapi
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

import lrmaCU.terra.table_utils as lrma_table_utils

bucket = os.environ['WORKSPACE_BUCKET']
workspace = os.environ['WORKSPACE_NAME']
namespace = os.environ['WORKSPACE_NAMESPACE']

flowcell_table = "sample"
sample_table = "sample_set"

#################################################################

print(f"Namespace: {namespace}")
print(f"Workspace: {workspace}")
print(f"Bucket:    {bucket}")
print()
print(f"Flowcell table: {flowcell_table}")
print(f"Sample table:   {sample_table}")

Namespace: broad-firecloud-dsde-methods
Workspace: sr-malaria
Bucket:    gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd

Flowcell table: sample
Sample table:   sample_set


***
## Load in Data

In [3]:
def load_table(namespace, workspace, table_name, store_membership=False, membership_column="samples"):
    ent_old = fapi.get_entities(namespace, workspace, table_name).json()
    tbl_old = None

    membership = None
    if len(ent_old) > 0:
        tbl_old = pd.DataFrame(list(map(lambda e: e['attributes'], ent_old)))
        tbl_old[f"entity:{table_name}_id"] = list(map(lambda f: f['name'], ent_old))

        if store_membership:
            membership = list(map(lambda g: set(map(lambda h: h['entityName'], g['items'])), tbl_old[membership_column]))
            del tbl_old[membership_column]

        c = list(tbl_old.columns)
        c.remove(f"entity:{table_name}_id")
        c = [f"entity:{table_name}_id"] + c
        tbl_old = tbl_old[c]
        tbl_old = tbl_old.astype(str)

    return tbl_old, membership

# We also have to remove any `nan` values here:
def fix_nans(df, quiet=True):
    if not quiet: print("Replacing all `nan` values with empty strings: ")
    for c in df.columns.values:
        nan_types = ("nan", float('nan'))
        has_nan = False
        num_denaned = 0
        for n in nan_types:
            if (sum(df[c] == n) > 0):
                num_denaned += sum(df[c] == n)
                df.loc[df[c] == n, c] = ""
                has_nan = True
        if has_nan and not quiet:
            print(f"\t{c}: {num_denaned}")

    if not quiet: print("Replacing numpy nan values...")
    if not quiet: print("Done.")
    return df.replace(np.nan, "")

In [4]:
# Get our flowcell and sample tables:
print("Loading Flowcell Table...\t", end="")
t_start = time.time()
tbl_fc, _ = load_table(namespace, workspace, flowcell_table)
t_end = time.time()
print(f"DONE\t(elapsed: {t_end-t_start:2.4f}s, {(t_end-t_start)/len(tbl_fc):8.6f}s/flowcell)")

print("Loading Sample Table...\t", end="")
t_start = time.time()
tbl_sm, _ = load_table(namespace, workspace, sample_table)
t_end = time.time()
print(f"DONE\t(elapsed: {t_end-t_start:2.4f}s, {(t_end-t_start)/len(tbl_sm):8.6f}s/sample)")

print("Cleaning NaN values from data...\t", end="")
tbl_fc = fix_nans(tbl_fc)
tbl_sm = fix_nans(tbl_sm)
print("DONE")

Loading Flowcell Table...	DONE	(elapsed: 19.5915s, 0.000841s/flowcell)
Loading Sample Table...	DONE	(elapsed: 18.6101s, 0.000885s/sample)
Cleaning NaN values from data...	DONE


In [10]:
# Find all new `flowcell` participants that are not in the `sample` table:
new_participant_set = set(tbl_fc['participant'].values) - set(tbl_sm["entity:sample_set_id"].values)

# Get all new entries in the `flowcell` table for these participants:
new_flowcells = tbl_fc[tbl_fc["participant"].isin(new_participant_set)]

# Get all new flowcells without data:
new_flowcells_unaligned = new_flowcells[new_flowcells["aligned_bam"] == ""]

print(f"Found {len(new_flowcells)} new data entries.")

if len(new_flowcells_unaligned) > 0:
    print()
    print("New flowcells with unaligned data:")
    print()
    display(new_flowcells_unaligned)
    print()
    print()
    
    # Set new flowcells to a value you can use going forward:
    new_flowcells = new_flowcells[~new_flowcells["participant"].isin(set(new_flowcells_unaligned['participant']))]
    
    # If there's no more data, set a bad value to `new_flowcells` so that everything fails:
    if len(new_flowcells) == 0:
        new_flowcells = None
    
    raise RuntimeError("There are flowcells without aligned data.  You must run `SRFlowcell` before moving flowcell data to the sample table.  You may continue, but the above rows will not be added.")


Found 980 new data entries.

New flowcells with unaligned data:



,entity:sample_id,collection_site,country,input_fq_end2,insert_size_standard_deviation,unaligned_bam,num_reads,num_reads_Q10,study,read_length,...,raw_est_fold_cov,aligned_bam,read_qual_median,pct_properly_paired_reads,collection_site_longitude,ENA,aligned_bai,average_identity,fq1,input_bai
22684,SEN_2023_BAN.PD2_0198,Bandafassi,Senegal,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...,,,,,...,,,,,-12.3106,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...
22777,SEN_2020_BAN.PPS_0025,,Senegal,,,,,,,,...,,,,,-12.3106,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...
22779,SEN_2020_BAN.PD1_0003,,Senegal,,,,,,,,...,,,,,-12.3106,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...
22800,SEN_2020_BAN.PPS_0032,,Senegal,,,,,,,,...,,,,,-12.3106,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...
22820,SEN_2020_BAN.PPS_0053,,Senegal,,,,,,,,...,,,,,-12.3106,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...
22889,SEN_2020_BAN.PPS_0027,,Senegal,,,,,,,,...,,,,,-12.3106,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...
22925,SEN_2020_BAN.PPS_0290,,Senegal,,,,,,,,...,,,,,-12.3106,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...
22930,SEN_2020_BAN.PPS_0119,,Senegal,,,,,,,,...,,,,,-12.3106,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...
23162,SEN_2020_SES.PD3_0120,,Senegal,,,,,,,,...,,,,,-16.358667,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...
23231,SEN_2020_BAN.PPS_0284,,Senegal,,,,,,,,...,,,,,-12.3106,,,,,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...


RuntimeError: There are flowcells without aligned data.  You must run `SRFlowcell` before moving flowcell data to the sample table.  You may continue, but the above rows will not be added.

In [11]:
print("New flowcells with aligned data:")
new_flowcells

New flowcells with aligned data:


,entity:sample_id,collection_site,country,input_fq_end2,insert_size_standard_deviation,unaligned_bam,num_reads,num_reads_Q10,study,read_length,...,raw_est_fold_cov,aligned_bam,read_qual_median,pct_properly_paired_reads,collection_site_longitude,ENA,aligned_bai,average_identity,fq1,input_bai
22321,IT,,,gs://broad-dsde-methods-long-reads/resources/n...,137.0,None,148292942.0,144901308.0,HQ Assemblies 2018 (Otto et al.),250.0,...,1588.8866116977879,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,22.704,86.1,,ERR862171,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,96.92505047322821,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,
22322,KH01,,,gs://broad-dsde-methods-long-reads/resources/n...,169.7,None,10888862.0,10888804.0,HQ Assemblies 2018 (Otto et al.),250.0,...,116.66885028435674,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,26.68,87.9,,ERR1015542,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,98.12356717878158,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,
22323,KH02,,,gs://broad-dsde-methods-long-reads/resources/n...,850.3,None,4143503.0,4143492.0,HQ Assemblies 2018 (Otto et al.),250.0,...,44.39561555282664,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,27.264,85.0,,ERR980503,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,97.94544917387023,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,
22324,SN01,,,gs://broad-dsde-methods-long-reads/resources/n...,775.1,None,5673948.0,5673927.0,HQ Assemblies 2018 (Otto et al.),250.0,...,60.79358795558483,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,27.48,87.5,,ERR980502,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,98.04197439049706,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,
22325,GN01,,,gs://broad-dsde-methods-long-reads/resources/n...,844.5,None,4390999.0,4390984.0,HQ Assemblies 2018 (Otto et al.),250.0,...,47.047414590226246,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,26.68,72.9,,ERR980507,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,97.94265090935644,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23296,SEN_2020_NDO.PD2_0054,,Senegal,,1440.7,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,30686695.0,30686695.0,,151.0,...,198.59096207709658,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,27.920529801324502,68.2,-14.102,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,97.56937002654374,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...
23297,SEN_2116_SES.PPS_0198,,Senegal,,352.3,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,14996464.0,14996456.0,,151.0,...,97.05060168631859,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,27.94701986754967,78.2,-16.358667,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,98.37101081187357,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...
23298,SEN_2020_NDO.PD3_0141,,Senegal,,1009.6,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,16776897.0,16776836.0,,151.0,...,108.5727907778389,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,17.920529801324502,69.4,-14.102,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,96.78916960797861,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...
23299,SEN_2020_NDO.PD3_0130,,Senegal,,1274.4,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,30357059.0,30357056.0,,151.0,...,196.4577010538666,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,27.350993377483444,61.1,-14.102,,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,98.7829236489792,gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/r...,gs://broad-dsp-lrma-sr-malaria/falciparum/sene...


In [12]:
# Set up a new dataframe in the style of the `sample` table:
new_sample_df = tbl_sm.iloc[:0,:].copy()

# Add in our new data:
metadata_to_port = [
    'collection_site',
    'collection_site_latitude',
    'collection_site_longitude',
    'country',
    'country_latitude',
    'country_longitude',
    'dataset_provenance',
    'year',
    'study',
    'sample_type',
    'ENA',
    'additional_metadata'
]

new_sample_df["entity:sample_set_id"] = new_flowcells["participant"]
for c in metadata_to_port:
    new_sample_df[c] = new_flowcells[c]

new_sample_df = fix_nans(new_sample_df)
new_sample_df

,entity:sample_set_id,barcode,year,country,aligned_num_reads,hc_rescored_vcf,aligned_read_length_mean,collection_site_latitude,dataset_provenance,hc_bamout,...,pct_properly_paired_reads,hc_raw_vcf,collection_site_longitude,ENA,aligned_bai,drug_res_report_expanded,func_annotated_vcf,hrp2,hrp3,additional_metadata
22321,IT,,,,,,,,HQ Assemblies 2018 (Otto et al.),,...,,,,ERR862171,,,,,,
22322,KH01,,,,,,,,HQ Assemblies 2018 (Otto et al.),,...,,,,ERR1015542,,,,,,
22323,KH02,,,,,,,,HQ Assemblies 2018 (Otto et al.),,...,,,,ERR980503,,,,,,
22324,SN01,,,,,,,,HQ Assemblies 2018 (Otto et al.),,...,,,,ERR980502,,,,,,
22325,GN01,,,,,,,,HQ Assemblies 2018 (Otto et al.),,...,,,,ERR980507,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23296,SEN_2020_NDO.PD2_0054,,2020,Senegal,,,,13.753,Broad 2020 Senegal Dataset,,...,,,-14.102,,,,,,,
23297,SEN_2116_SES.PPS_0198,,2116,Senegal,,,,14.920131,Broad 2020 Senegal Dataset,,...,,,-16.358667,,,,,,,
23298,SEN_2020_NDO.PD3_0141,,2020,Senegal,,,,13.753,Broad 2020 Senegal Dataset,,...,,,-14.102,,,,,,,
23299,SEN_2020_NDO.PD3_0130,,2020,Senegal,,,,13.753,Broad 2020 Senegal Dataset,,...,,,-14.102,,,,,,,


## Upload New Sample/Flowcell Data

In [13]:
def upload_table(namespace, workspace, table, label):
    # upload new samples
    print(f"Uploading data from {label} ... ", end="")
    a = fapi.upload_entities(namespace, workspace, entity_data=table.to_csv(index=False, sep="\t"), model='flexible')

    if a.status_code == 200:
        print(f'Uploaded {len(table)} rows successfully.')
    else:
        print(a.json())

In [14]:
# Now let's upload the new sample info.  We'll add the links to the `flowcell` table next.
if len(new_sample_df) > 0:
    upload_table(namespace, workspace, new_sample_df, f"Sample table ({sample_table})")

Uploading data from Sample table (sample_set) ... Uploaded 969 rows successfully.


In [15]:
# Now fix all the references in the `sample` table to link to the new participants:
if len(new_sample_df) > 0:

    print("Fixing sample table `sample` entries for links to the flowcells:")

    # "{'itemsType': 'EntityReference', 'items': [{'entityType': 'sample_set', 'entityName': 'PG0004-CW'}]}"
    for participant in tqdm(new_sample_df['entity:sample_set_id'].values, desc="Updating sample flowcell refs"):

        flowcell_ids = tbl_fc[tbl_fc["participant"] == participant]["entity:sample_id"].values

        print(f"\t{participant} -> {list(flowcell_ids)}\t", end="")

        # Update flowcell info:
        lrma_table_utils.fill_in_entity_members(ns=namespace, 
                                                ws=workspace, 
                                                etype="sample_set",
                                                ename=participant,
                                                member_entity_type="sample",
                                                members=flowcell_ids,
                                                operation=lrma_table_utils.MembersOperationType.RESET
                                               )
        print("DONE")
    print(f"Updated {len(new_sample_df)} sample -> flowcell references.")

Fixing sample table `sample` entries for links to the flowcells:


Updating sample flowcell refs:   0%|          | 0/969 [00:00<?, ?it/s]

	IT -> ['IT']	DONE
	KH01 -> ['KH01']	DONE
	KH02 -> ['KH02']	DONE
	SN01 -> ['SN01']	DONE
	GN01 -> ['GN01']	DONE
	GA01 -> ['GA01']	DONE
	SEN_2022_BAN.PD1_0213 -> ['SEN_2022_BAN.PD1_0213']	DONE
	SEN_2022_BAN.PD1_0237 -> ['SEN_2022_BAN.PD1_0237']	DONE
	SEN_2022_BAN.PD2_0151 -> ['SEN_2022_BAN.PD2_0151']	DONE
	SEN_2022_BAN.PD2_0169 -> ['SEN_2022_BAN.PD2_0169']	DONE
	SEN_2022_BAN.PD2_0170 -> ['SEN_2022_BAN.PD2_0170']	DONE
	SEN_2022_BAN.PD2_0172 -> ['SEN_2022_BAN.PD2_0172']	DONE
	SEN_2022_BAN.PD2_0173 -> ['SEN_2022_BAN.PD2_0173']	DONE
	SEN_2022_BAN.PD2_0174 -> ['SEN_2022_BAN.PD2_0174']	DONE
	SEN_2022_BAN.PD2_0177 -> ['SEN_2022_BAN.PD2_0177']	DONE
	SEN_2022_BAN.PD2_0179 -> ['SEN_2022_BAN.PD2_0179']	DONE
	SEN_2022_BAN.PD2_0180 -> ['SEN_2022_BAN.PD2_0180']	DONE
	SEN_2022_BAN.PD2_0182 -> ['SEN_2022_BAN.PD2_0182']	DONE
	SEN_2022_BAN.PD2_0183 -> ['SEN_2022_BAN.PD2_0183']	DONE
	SEN_2022_BAN.PD2_0184 -> ['SEN_2022_BAN.PD2_0184']	DONE
	SEN_2022_BAN.PD2_0188 -> ['SEN_2022_BAN.PD2_0188']	DONE
	SEN_2022_B

	SEN_2022_KDG.CXX_0122 -> ['SEN_2022_KDG.CXX_0122']	DONE
	SEN_2022_MAD.SXX_0184 -> ['SEN_2022_MAD.SXX_0184']	DONE
	SEN_2022_MAD.SXX_0185 -> ['SEN_2022_MAD.SXX_0185']	DONE
	SEN_2022_MAD.SXX_0186 -> ['SEN_2022_MAD.SXX_0186']	DONE
	SEN_2022_MAD.SXX_0187 -> ['SEN_2022_MAD.SXX_0187']	DONE
	SEN_2022_MAD.SXX_0189 -> ['SEN_2022_MAD.SXX_0189']	DONE
	SEN_2022_MAD.SXX_0190 -> ['SEN_2022_MAD.SXX_0190']	DONE
	SEN_2022_MAD.SXX_0194 -> ['SEN_2022_MAD.SXX_0194']	DONE
	SEN_2022_MAD.SXX_0197 -> ['SEN_2022_MAD.SXX_0197']	DONE
	SEN_2022_MAD.SXX_0199 -> ['SEN_2022_MAD.SXX_0199']	DONE
	SEN_2022_MAD.SXX_0202 -> ['SEN_2022_MAD.SXX_0202']	DONE
	SEN_2022_MAD.SXX_0217 -> ['SEN_2022_MAD.SXX_0217']	DONE
	SEN_2022_MAD.SXX_0219 -> ['SEN_2022_MAD.SXX_0219']	DONE
	SEN_2022_MAD.SXX_0220 -> ['SEN_2022_MAD.SXX_0220']	DONE
	SEN_2022_MAD.SXX_0221 -> ['SEN_2022_MAD.SXX_0221']	DONE
	SEN_2022_MAD.SXX_0222 -> ['SEN_2022_MAD.SXX_0222']	DONE
	SEN_2022_MAD.SXX_0223 -> ['SEN_2022_MAD.SXX_0223']	DONE
	SEN_2022_MAD.SXX_0227 -> ['SEN

	SEN_2022_SES.PD2_0095 -> ['SEN_2022_SES.PD2_0095']	DONE
	SEN_2022_SES.PD2_0097 -> ['SEN_2022_SES.PD2_0097']	DONE
	SEN_2022_SES.PD2_0111 -> ['SEN_2022_SES.PD2_0111']	DONE
	SEN_2022_SES.PD2_0113 -> ['SEN_2022_SES.PD2_0113']	DONE
	SEN_2022_SES.PD2_0115 -> ['SEN_2022_SES.PD2_0115']	DONE
	SEN_2022_SES.PD2_0116 -> ['SEN_2022_SES.PD2_0116']	DONE
	SEN_2022_SES.PD2_0120 -> ['SEN_2022_SES.PD2_0120']	DONE
	SEN_2022_SES.PD2_0122 -> ['SEN_2022_SES.PD2_0122']	DONE
	SEN_2022_SES.PD2_0127 -> ['SEN_2022_SES.PD2_0127']	DONE
	SEN_2022_SES.PD2_0132 -> ['SEN_2022_SES.PD2_0132']	DONE
	SEN_2022_SES.PD2_0134 -> ['SEN_2022_SES.PD2_0134']	DONE
	SEN_2022_SES.PD2_0135 -> ['SEN_2022_SES.PD2_0135']	DONE
	SEN_2022_SES.PD2_0136 -> ['SEN_2022_SES.PD2_0136']	DONE
	SEN_2022_SES.PD2_0137 -> ['SEN_2022_SES.PD2_0137']	DONE
	SEN_2022_SES.PD2_0138 -> ['SEN_2022_SES.PD2_0138']	DONE
	SEN_2022_SES.PD2_0142 -> ['SEN_2022_SES.PD2_0142']	DONE
	SEN_2022_SES.PD2_0143 -> ['SEN_2022_SES.PD2_0143']	DONE
	SEN_2022_SES.PD2_0145 -> ['SEN

	SEN_2020_SES.PPS_0125 -> ['SEN_2020_SES.PPS_0125']	DONE
	SEN_2020_SES.PPS_0133 -> ['SEN_2020_SES.PPS_0133']	DONE
	SEN_2020_SES.PPS_0140 -> ['SEN_2020_SES.PPS_0140']	DONE
	SEN_2020_SES.PPS_0149 -> ['SEN_2020_SES.PPS_0149']	DONE
	SEN_2020_SES.PPS_0157 -> ['SEN_2020_SES.PPS_0157']	DONE
	SEN_2020_SES.PPS_0165 -> ['SEN_2020_SES.PPS_0165']	DONE
	SEN_2020_SES.PPS_0173 -> ['SEN_2020_SES.PPS_0173']	DONE
	SEN_2020_SES.PPS_0181 -> ['SEN_2020_SES.PPS_0181']	DONE
	SEN_2020_SES.PPS_0127 -> ['SEN_2020_SES.PPS_0127']	DONE
	SEN_2020_SES.PPS_0197 -> ['SEN_2020_SES.PPS_0197']	DONE
	SEN_2019_TAO.SXX_0009 -> ['SEN_2019_TAO.SXX_0009']	DONE
	SEN_2020_SES.PPS_0118 -> ['SEN_2020_SES.PPS_0118']	DONE
	SEN_2020_SES.PPS_0126 -> ['SEN_2020_SES.PPS_0126']	DONE
	SEN_2020_SES.PPS_0134 -> ['SEN_2020_SES.PPS_0134']	DONE
	SEN_2020_SES.PPS_0142 -> ['SEN_2020_SES.PPS_0142']	DONE
	SEN_2020_SES.PPS_0150 -> ['SEN_2020_SES.PPS_0150']	DONE
	SEN_2020_SES.PPS_0158 -> ['SEN_2020_SES.PPS_0158']	DONE
	SEN_2020_SES.PPS_0166 -> ['SEN

	SEN_2020_BAN.PPS_0052 -> ['SEN_2020_BAN.PPS_0052']	DONE
	SEN_2020_BAN.PD1_0030 -> ['SEN_2020_BAN.PD1_0030']	DONE
	SEN_2020_BAN.PD1_0184 -> ['SEN_2020_BAN.PD1_0184']	DONE
	SEN_2020_BAN.PPS_0005 -> ['SEN_2020_BAN.PPS_0005']	DONE
	SEN_2020_BAN.PD2_0213 -> ['SEN_2020_BAN.PD2_0213']	DONE
	SEN_2020_BAN.PD1_0019 -> ['SEN_2020_BAN.PD1_0019']	DONE
	SEN_2020_BAN.PD1_0104 -> ['SEN_2020_BAN.PD1_0104']	DONE
	SEN_2020_BAN.PD1_0007 -> ['SEN_2020_BAN.PD1_0007']	DONE
	SEN_2020_BAN.PD2_0168 -> ['SEN_2020_BAN.PD2_0168']	DONE
	SEN_2020_BAN.PPS_0157 -> ['SEN_2020_BAN.PPS_0157']	DONE
	SEN_2020_BAN.PD1_0035 -> ['SEN_2020_BAN.PD1_0035']	DONE
	SEN_2020_BAN.PPS_0139 -> ['SEN_2020_BAN.PPS_0139']	DONE
	SEN_2020_BAN.PD2_0123 -> ['SEN_2020_BAN.PD2_0123']	DONE
	SEN_2020_BAN.PPS_0120 -> ['SEN_2020_BAN.PPS_0120']	DONE
	SEN_2020_BAN.PD2_0109 -> ['SEN_2020_BAN.PD2_0109']	DONE
	SEN_2020_BAN.PPS_0099 -> ['SEN_2020_BAN.PPS_0099']	DONE
	SEN_2020_BAN.PD2_0219 -> ['SEN_2020_BAN.PD2_0219']	DONE
	SEN_2020_BAN.PPS_0062 -> ['SEN

	SEN_2020_SES.PD1_0069 -> ['SEN_2020_SES.PD1_0069']	DONE
	SEN_2020_SES.PD3_0015 -> ['SEN_2020_SES.PD3_0015']	DONE
	SEN_2020_SES.PD1_0076 -> ['SEN_2020_SES.PD1_0076']	DONE
	SEN_2020_SES.PD2_0057 -> ['SEN_2020_SES.PD2_0057']	DONE
	SEN_2020_SES.PD2_0018 -> ['SEN_2020_SES.PD2_0018']	DONE
	SEN_2020_SES.PD1_0036 -> ['SEN_2020_SES.PD1_0036']	DONE
	SEN_2020_SES.PD1_0046 -> ['SEN_2020_SES.PD1_0046']	DONE
	SEN_2020_SES.PD1_0055 -> ['SEN_2020_SES.PD1_0055']	DONE
	SEN_2020_SES.PD1_0080 -> ['SEN_2020_SES.PD1_0080']	DONE
	SEN_2020_SES.PD1_0089 -> ['SEN_2020_SES.PD1_0089']	DONE
	SEN_2020_SES.PD1_0116 -> ['SEN_2020_SES.PD1_0116']	DONE
	SEN_2020_SES.PD2_0035 -> ['SEN_2020_SES.PD2_0035']	DONE
	SEN_2020_SES.PD2_0058 -> ['SEN_2020_SES.PD2_0058']	DONE
	SEN_2020_SES.PD3_0070 -> ['SEN_2020_SES.PD3_0070']	DONE
	SEN_2020_SES.PD3_0080 -> ['SEN_2020_SES.PD3_0080']	DONE
	SEN_2020_SES.PD1_0039 -> ['SEN_2020_SES.PD1_0039']	DONE
	SEN_2020_SES.PD1_0047 -> ['SEN_2020_SES.PD1_0047']	DONE
	SEN_2020_SES.PD1_0056 -> ['SEN

	SEN_2020_NDO.PD3_0145 -> ['SEN_2020_NDO.PD3_0145']	DONE
	SEN_2019_ROB.SXX_0008 -> ['SEN_2019_ROB.SXX_0008']	DONE
	SEN_2019_ROB.SXX_0199 -> ['SEN_2019_ROB.SXX_0199']	DONE
	SEN_2020_SES.PD3_0095 -> ['SEN_2020_SES.PD3_0095']	DONE
	SEN_2019_CSS.SXX_0005 -> ['SEN_2019_CSS.SXX_0005']	DONE
	SEN_2020_SES.PD3_0112 -> ['SEN_2020_SES.PD3_0112']	DONE
	SEN_2019_CSS.SXX_0013 -> ['SEN_2019_CSS.SXX_0013']	DONE
	SEN_2020_SES.PD3_0153 -> ['SEN_2020_SES.PD3_0153']	DONE
	SEN_2019_CSS.SXX_0021 -> ['SEN_2019_CSS.SXX_0021']	DONE
	SEN_2020_NDO.PPS_0115 -> ['SEN_2020_NDO.PPS_0115']	DONE
	SEN_2019_CSS.SXX_0029 -> ['SEN_2019_CSS.SXX_0029']	DONE
	SEN_2020_NDO.PD1_0130 -> ['SEN_2020_NDO.PD1_0130']	DONE
	SEN_2019_TAO.SXX_0004 -> ['SEN_2019_TAO.SXX_0004']	DONE
	SEN_2020_NDO.PD1_0198 -> ['SEN_2020_NDO.PD1_0198']	DONE
	SEN_2019_ROS.SXX_0002 -> ['SEN_2019_ROS.SXX_0002']	DONE
	SEN_2019_ROB.SXX_0001 -> ['SEN_2019_ROB.SXX_0001']	DONE
	SEN_2019_GMK.SXX_0003 -> ['SEN_2019_GMK.SXX_0003']	DONE
	SEN_2019_ROB.SXX_0009 -> ['SEN

***
***
***